In [1]:
from typing import Sequence, Optional, Iterator

import clingo

In [2]:
def solve(programs,
          ctl: Optional[clingo.Control] = None,
          parts=(('base', ()),),
          context=None,
          report=False,
          report_models=True,
          report_result=True,
          symbol_sep=' ',
          model_sep='\n'
          ) -> Iterator[Sequence[clingo.Symbol]]:
    if ctl is None:
        ctl = clingo.Control()
        ctl.configuration.solve.models = 0
    ctl.add('base', [], '\n'.join(programs))
    ctl.ground(parts, context=context)
    with ctl.solve(yield_=True) as solve_handle:
        models = 0
        for model in solve_handle:
            symbols = sorted(model.symbols(shown=True))
            if report and report_models:
                print("Answer {}:".format(model.number), end=' ')
                print("{",
                      symbol_sep.join(map(str, symbols)), "}", sep=symbol_sep, end=model_sep)
            models += 1
            yield symbols
        if report and report_result:
            solve_result = solve_handle.get()
            print(solve_result, end='')
            if solve_result.satisfiable:
                print(" {}{}".format(models, '' if solve_result.exhausted else '+'))
            else:
                print()

In [3]:
primal = """

%! trace_rule {"a :- k, not b."}
a :- k, not b.
%! trace_rule {"c :- a, b."}
c :- a, b.
%! trace_rule {"c :- k."}
c :- k.
%! trace_rule {"k :- e, not b."}
k :- e, not b.

%! trace_rule {"e."}
e.

%! trace_rule {"b :- not a."}
b :- not a.
%! trace_rule {"f :- e, not k, not c."}
f :- e, not k, not c.

%!show_trace a.
%!show_trace c.
%!show_trace k.

%!show_trace e.

%!show_trace b.
%!show_trace f.

"""

primal_answer_sets = tuple(solve([primal], report=True))

Answer 1: { b e f }
Answer 2: { a c e k }
SAT 2


In [4]:
! xclingo -n 0 0 "P01_b_primal.lp"

Answer 1
  *
  |__b :- not a.

  *
  |__f :- e, not k, not c.
  |  |__e.

  *
  |__e.

Answer 2
  *
  |__e.

  *
  |__k :- e, not b.
  |  |__e.

  *
  |__a :- k, not b.
  |  |__k :- e, not b.
  |  |  |__e.

  *
  |__c :- k.
  |  |__k :- e, not b.
  |  |  |__e.

